In [1]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    RouterQueryEngine,
    OpenAIAgent,
    QueryEngineTool,
    FunctionTool,
} from "npm:llamaindex@0.1.18"
const keys = await mod.load({export:true}) // read API key from .env

In [3]:
const OPENAI_API_KEY=
const apiKey=

In [4]:
const documents1 = await new SimpleDirectoryReader().loadData({directoryPath: "./data"})
const index1 = await VectorStoreIndex.fromDocuments(documents1)
const queryEngine1 = index1.asQueryEngine()

In [6]:
const documents2 = await new SimpleDirectoryReader().loadData({directoryPath: "./data2"})
const index2 = await VectorStoreIndex.fromDocuments(documents2)
const queryEngine2=index2.asQueryEngine()

In [7]:
let response1 = await queryEngine1.query({query:"What did author did before college?"}); 
let response2 = await queryEngine2.query({query:"What is React"}); 

In [10]:
console.log(response1.response)
console.log(response2.response)

Before college, the author finished high school in Russia in 2009.
React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React allows for the development of dynamic and interactive user interfaces for web applications. It has undergone significant updates and improvements over the years, with features like React Fiber for improved rendering performance and React Native for native mobile app development.


In [14]:
const queryEngine = await RouterQueryEngine.fromDefaults({
    queryEngineTools: [
      {
        queryEngine: queryEngine1,
        description: "Useful for questions about Dan Abramov",
      },
      {
        queryEngine: queryEngine2,
        description: "Useful for questions about the React library",
      },
    ],
  });

In [15]:
let response3 = await queryEngine.query({query: "What is react?"});
let response4 = await queryEngine.query({query: "What did Dan Abramov do in colllege?"});

In [16]:
console.log(response3.response);
console.log(response4.response);

React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create dynamic and interactive user interfaces for web applications. It has undergone significant updates and improvements over the years, with features like React Fiber, which introduced a new set of internal algorithms for rendering, and React 16.0 and React 18, which brought further enhancements and new features to the library.
In college, Dan Abramov slacked off, skipped lectures, and eventually stopped attending altogether. He also returned his Macbook and did not show up for midterm exams.


In [24]:
function sumNumbers({a,b}) {
    return a + b;
  }

In [25]:
const sumJSON = {
    type: "object",
    properties: {
      a: {
        type: "number",
        description: "The first number",
      },
      b: {
        type: "number",
        description: "The second number",
      },
    },
    required: ["a", "b"],
  };

In [26]:
const sumFunctionTool = new FunctionTool(sumNumbers, {
    name: "sumNumbers",
    description: "Use this function to sum two numbers",
    parameters: sumJSON,
  });

In [27]:
const queryEngineTool = new QueryEngineTool({
    queryEngine: queryEngine,
    metadata: {
        name: "react_and_dan_abramov_engine",
        description: "A tool that can answer questions about Dan Abramov and React",
    },
});

In [21]:
const agent = new OpenAIAgent(
    {
        tools: [sumFunctionTool,queryEngineTool],
        verbose: true
    }
)

In [29]:
let response5 = await agent.chat({message:"What is React? Use a tool."})
console.log(response5.response)

Added user message to memory!: What is React? Use a tool.
=== Calling Function ===
Calling function: react_and_dan_abramov_engine with args: {
"query": "What is React?"
}
Got output React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create dynamic and interactive user interfaces for web applications. It has undergone significant updates and improvements over the years, including the introduction of React Fiber, a new set of internal algorithms for rendering, and the release of React 16.0 in September 2017. Additionally, React 18, released in March 2022, introduced a new concurrent renderer, automatic batching, and support for server-side rendering with Suspense.
React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 and has since gained popularity for its ability to create dynamic an

In [30]:
let response6 = await agent.chat({message:"What is 501 + 5?"})
console.log(response6.response)

Added user message to memory!: What is 501 + 5?
=== Calling Function ===
Calling function: sumNumbers with args: {
"a": 501,
"b": 5
}
Got output 506
The sum of 501 and 5 is 506.
